# Create Jupyter Book
 <span style="color:red">**Note**:</span> **Windows users** using **Python 3.8** might be impacted by this known [issue](https://jupyterbook.org/advanced/advanced.html#working-on-windows) on the latest **jupyter book** version.

  <span style="color:red">**Note**:</span> In Jupyter Book versions above 0.7.0, verify that the names of the files do not start with a **number**. Due to the new design of jupyter book, the prefix number will be removed from the file name in the Table of Contents. Please refer to this [section](https://jupyterbook.org/customize/toc.html#automatically-generate-your-toc-yml-file) of the documentation for more information.


This is a notebook to create a [Jupyter Book](https://jupyterbook.org/intro.html), which is an organized collections of Jupyter notebooks.

The best way to use this notebook is by clicking Run all in the toolbar above. This will run all the cells in a step-by-step order so that you can create your Jupyter Book.

 
## 1. Installation

To install the Jupyter Book command-line interface (CLI), use `pip`!

In [ ]:
import sys

def getJupyterBookVersion():
    cmd = f'{sys.executable} -m pip show jupyter-book'
    cmdOutput = !{cmd}
    if len(cmdOutput) <= 1:
        !pip install jupyter-book
        cmd = f'{sys.executable} -m pip show jupyter-book'
        cmdOutput = !{cmd}
    for x in cmdOutput:
        if 'version' in x.lower():
            version_str = tuple(x.split(': ')[1].replace('.', ''))
            version = tuple(int(x) for x in version_str)
    return version, version_str

In [ ]:
import sys

try:
    version, version_str = getJupyterBookVersion()
    if version:
        display_version = '.'.join(version_str)
        print(f'Jupyter-book version {display_version} is already installed!')
        use_current = input(f'Would you like to create a book using the current installed version [{display_version}] of jupyter-book? [y/n]').lower()

        while use_current not in ['','yes', 'y', 'no', 'n']:
            use_current = input(f'Would you like to create a book using the current installed version [{display_version}] of jupyter-book? [y/n]').lower()
        if use_current in ['no', 'n']:
            install_version = input(f'Please enter the version you would like to use (eg. 0.7.4)')
            if install_version:
                !pip uninstall jupyter-book --yes
                version = ''
                print('Installing Jupyter-book...')
                !pip install jupyter-book=="$install_version"
                version, _ = getJupyterBookVersion()
except Exception as e:
    raise SystemExit(str(e))

## 2. Create a new book

Create a book using your own notebooks and markdown pages:

<span style="color:red">Note:</span> Notebook and markdown filenames cannot contain spaces

In [ ]:
import os, re, subprocess
from sys import platform
import shutil

os.environ["LC_ALL"]="en_US.UTF-8"

try:
    overwrite = False
    book_name = input('Please provide the path where the book needs to be saved along with the book name ex-> D:\Book1: ') 

    if (os.path.exists(book_name)):
        new_book_name = input('A folder named ' + book_name + ' already exists. Enter a new name or the same name to overwrite the existing folder.\n')
        if book_name == new_book_name:
            overwrite = True
        book_name = new_book_name

    content_folder = input('Please provide the path to your folder containing notebooks and markdown files: ')

    while (not os.path.exists(content_folder)):
        content_folder = input('Cannot find folder ' + content_folder + '. Please provide another path: ')
    if version and version < (0,7,0):
        if overwrite:
            !jupyter-book create "$book_name" --content-folder "$content_folder" --overwrite
        else:
            !jupyter-book create "$book_name" --content-folder "$content_folder"
    elif version:
        if overwrite:
            shutil.rmtree(book_name)
            # waiting for shutil to remove the directory
            while os.path.exists(book_name):
                pass
        os.mkdir(book_name)
        content_folder = os.path.join(content_folder,'')
        if platform in ['linux', 'darwin']:
            subprocess.check_call(['cp', '-r',content_folder, book_name])
        else:
            subprocess.check_call(['xcopy', content_folder, book_name, '/E', '/H'])
        !jupyter-book toc "$book_name"
except Exception as e:
    raise SystemExit(str(e))

## 3. Format toc.yml file of the book

Create a valid toc.yml file that enables your book to be opened on our viewlet.

<span style="color:red">Note:</span> Skipping this step may result in a malformed book being generated.

In [ ]:
# Update toc file, book title and clean up the directores
import shutil
from os import path
import yaml
def getMarkdownFile(url):
    # url are usually defined in toc as: "- url: <Path\to\notebookFile>"
    # substring from 7th postion excluding the "- url: " from the path
    #folders = url[7:].rstrip().split(path.sep)
    if (os.name == 'nt'):
        # if windows get the drive letter and add it to the url
        driveletter = content_folder.split(path.sep)[0]
        markdownFilePath = path.join(driveletter, url[7:].rstrip()+'.md')
    else:
        markdownFilePath = url[7:].rstrip() + '.md'
    return markdownFilePath

def getAllSections(visited, data, current):
    if current not in visited:
        visited.append(current)
        for x in range(len(current)):
            if 'sections' in current[x].keys():
                tmp = {}
                tmp['title'] = path.dirname(current[x]['file']).title()
                tmp['file'] = current[x]['file']
                tmp['numbered'] = False
                tmp['expand_sections'] = True
                tmp['sections'] = current[x]['sections']
                current[x] = tmp
                getAllSections(visited, data, current[x]['sections'])
            else:
                current[x]['title'] = path.basename(current[x]['file']).title()

if version < (0,7,0):
    tocFilePath = path.join(book_name, "_data", "toc.yml")
    f = open(tocFilePath, "r")
    title = ''
    replacedString = ''
    result = f.read()
    f.close()
    contentFolders = []

    firstLevelUrls = re.findall(r'^(?:\s+$[\r\n]+)+(\- url: [a-zA-Z0-9\\.\s\-\/]+$[\r\n]+)', result, re.MULTILINE)
    urls = re.findall(r'- url: [a-zA-Z0-9\\.\s\-\/]+$', result, re.MULTILINE)
    headers = re.findall(r'- header: [a-zA-Z0-9\\.\s-]+$', result, re.MULTILINE)
    # all the markdown urls are placed at the end of the list. 
    possibleMarkdowns = re.findall(r'(- url: [a-zA-Z0-9\\.\s\-\/]+$[\r\n]+)[\r\n]', result, re.MULTILINE)

    try:
        if (firstLevelUrls or headers or urls):
            if (firstLevelUrls and len(firstLevelUrls) == 1):
                for url in firstLevelUrls:
                    # check first link is the markdown
                    title = url[url.rindex(path.sep)+1:].rstrip()
                    markdownFilePath = getMarkdownFile(url)
                    rootmarkdownExists = path.exists(markdownFilePath)
                    markdownUrl = urls[len(urls) -1]
                    if (rootmarkdownExists):
                        markdownUrl = url
                        replacedString = "\n- title: %s\n  url: /%s\n  not_numbered: true\n" % (title, title)
                    # else check if the last link in the url list is markdown    
                    elif (not headers and path.exists(getMarkdownFile(markdownUrl))):
                        rootmarkdownExists = True
                        title = markdownUrl[markdownUrl.rindex(path.sep)+1:].rstrip()
                        replacedString = "\n- title: %s\n  url: /%s\n  not_numbered: true\n  expand_sections: true\n  sections:  %s" % (title, title, url)
                        result = result.replace(markdownUrl, '')
                    # if there not markdowns and folders contains markdowns, handle them
                    elif (possibleMarkdowns):
                        markdownUrl = possibleMarkdowns[0]
                        title = markdownUrl[markdownUrl.rindex(path.sep)+1:].rstrip()
                        markdownFilePath = getMarkdownFile(markdownUrl)
                        rootmarkdownExists = path.exists(markdownFilePath)
                        if (rootmarkdownExists):
                            replacedString = "\n- title: %s\n  url: /%s\n  not_numbered: true\n  expand_sections: true\n  sections:  %s" % (title, title, url)
                            result = result.replace(markdownUrl, '')
                    # there is no markdown and we're adding the first link as is
                    else:
                        markdownUrl = url
                        replacedString = "\n- title: %s\n  url: /%s\n  not_numbered: true\n" % (title, title)
                    result = result.replace(url, replacedString)
            # Folders and handling them -> each header is a folder
            if (headers):
                for header in headers:
                    title = header[10:].rstrip()
                    # filters all the urls with /headerName in them
                    filtered = list(filter(lambda x: ("%s%s%s" % (path.sep, title.lower(), path.sep)) in x.lower(), urls))
                    index = urls.index(filtered[len(filtered)-1])
                    markdownFilePath = getMarkdownFile(filtered[len(filtered)-1])
                    markdownExists = path.exists(markdownFilePath)
                    if (not markdownExists):
                        index = urls.index(filtered[0])
                    folderEndIndex = urls[index].rindex(path.sep)
                    caseSensitiveFolderName = urls[index][urls[index].rindex(path.sep, 0, folderEndIndex)+1:folderEndIndex]
                    contentFolders.append(caseSensitiveFolderName)
                    urlValue = urls[index][urls[index].rindex(path.sep)+1:].rstrip()
                    replacedString = "\n- title: %s\n  url: /%s/%s\n%s  expand_sections: true\n  sections:  " % (title, caseSensitiveFolderName, urlValue, '  not_numbered: true\n' if markdownExists else '')
                    result = result.replace(header, replacedString)
                    if (markdownExists):
                        result = result.replace(urls[index], '')
                        del urls[index]
                if (urls):
                    for url in urls:
                        title = url[url.rindex(path.sep)+1:].rstrip()
                        urlValue = title
                        if (len(contentFolders) > 0):
                            folders = url[7:].split(path.sep)
                            if (folders[len(folders)-2] in contentFolders):
                                parentFolder = contentFolders.index(folders[len(folders)-2])
                                urlValue = "%s/%s" % (contentFolders[parentFolder], title)
                                replacedString = "\n  - title: %s\n    url: /%s" % (title, urlValue)
                            else:
                                replacedString = "\n  - title: %s\n    url: /%s" % (title, urlValue) if rootmarkdownExists else "\n- title: %s\n  url: /%s" % (title, urlValue)
                        result = result.replace(url, replacedString)
                    fwrite = open(tocFilePath, "w")
                    fwrite.write(result)
                    fwrite.close()
            # formattinf any left over urls in the file
            elif (urls):
                for url in urls:
                    title = url[url.rindex(path.sep)+1:].rstrip()
                    urlValue = title
                    replacedString = "\n  - title: %s\n    url: /%s" % (title, urlValue) if rootmarkdownExists else "\n- title: %s\n  url: /%s" % (title, urlValue)
                    result = result.replace(url, replacedString)
            fwrite = open(tocFilePath, "w")
            fwrite.write(result)
            fwrite.close()
        else:
            raise SystemExit(f'\n File Name contains unsupported-characters (ex: underscores) by Jupyter Book.\n')
        # Update the Book title in config file
        configFilePath = path.join(book_name, "_config.yml")
        f = open(configFilePath, "r")
        result = f.read()
        f.close()
        titleLine = re.search(r'title: [a-zA-Z0-9\\.\s\-\/]+$', result, re.MULTILINE).group()
        title = 'title: %s' % (path.splitext(path.basename(book_name))[0])
        result = result.replace(titleLine, title)
        fwrite = open(configFilePath, "w")
        fwrite.write(result)
        fwrite.close()
        # cleanup the directories
        with os.scandir(book_name) as root_dir:
            for path in root_dir:
                if path.is_file() and path.name not in ('_config.yml'):
                    os.remove(path)
                if path.is_dir() and path.name not in ('_data', 'content'):
                    shutil.rmtree(path)
    except Exception as e:
        raise SystemExit(str(e))
else:
    tocFilePath = path.join(book_name, "_toc.yml")
    configFilePath = path.join(book_name, "_config.yml")
    visited = []
    # modify generated toc file
    with open(tocFilePath, 'r') as stream:
        data = yaml.safe_load(stream)
        if not isinstance(data, list):
            new_data = []
            for k in data:
                if k == 'file':
                    new_data.append({k : data[k]})
                elif k == 'sections':
                    new_data[-1].update({'numbered' : 'false'})
                    new_data[-1].update({'expand_sections' : 'true'})
                    new_data[-1].update({k : data[k]})
            data = new_data
        for k in data:
            if 'sections' in k:
                getAllSections([], data, k['sections'])

    with open(tocFilePath, 'w') as outfile:
        yaml.dump(data, outfile, sort_keys=False)
    # create config file
    config = {}
    config['title'] = path.basename(book_name).title()
    with open(configFilePath, 'w') as output:
        yaml.dump(config, output, default_flow_style=False)


## 4. Open your Book!
**Run the below cell and click on the link to view your book in Azure Data Studio.**

In [ ]:
import re, os
from IPython.display import *
if os.name == 'nt':
    bookPath = book_name.replace('\\', '\\\\')
    display(HTML("<h2><b><a href=\"command:bookTreeView.openBook?&quot;"+str(bookPath)+"&quot;\"><font size=\"3\">Click here to open your Book in ADS</font></a></b></h2>"))
else:
    display(HTML("<h2><b><a href=\"command:bookTreeView.openBook?&quot;"+str(book_name)+"&quot;\"><font size=\"3\">Click here to open your Book in ADS</font></a></b></h2>"))

<span style="color:red">**Note**: On clicking the above link, we create a temporary toc.yml file for your convenience.</span>

**For versions < 0.7.0**

 Please update that file inside your book (located at: *YourbookPath*/_data/toc.yml) if you want to further customize your book following 
 instructions at https://legacy.jupyterbook.org/guide/01-5_tour.html#Table-of-Contents.

**For the newer versions**

Please update the _toc.yml file inside your book (located at: YourbookPath/_toc.yml). And refer to the documentation at https://jupyterbook.org/customize/toc.html for further customization.

In [ ]:
display(HTML("<h1><b>That's it!</b></h1><br/><p>You are good to view your book in Azure Data Studio by clicking on the above link.</p>"))